# Generate a sample of SN host galaxies

Contact author: Melissa Graham

Date last verified to run: Fri Feb 2 2024

RSP environment version: Weekly 2024_04

**Summary:**
A demo of how to get data for a subset of simulated DC2 galaxies which were designated as hosts for simulated SNe,
and to find out if these galaxies were detected in a deepCoadd image and are in the `Object` table.
This notebook does not explore whether the simulated SNe themselves were detected and are in the `diaObject` table.

Import packages and then instantiate the TAP service for DP0.2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.rsp import get_tap_service

In [ ]:
service = get_tap_service("tap")

Use a test area with a 0.5 deg radius in the central region of DC2.

From the `TruthSummary` table, return the list of all simulated objects in the test area (galaxies, stars, and SNe).

Join with the `MatchesTruth` table to retrive the `match_objectId`, if the simulated object was detected in a deepCoadd.

In [ ]:
query = '''SELECT ts.ra, ts.dec, ts.id, ts.id_truth_type, ts.host_galaxy, ts.truth_type, mt.match_objectId 
           FROM dp02_dc2_catalogs.TruthSummary AS ts  
           JOIN dp02_dc2_catalogs.MatchesTruth AS mt ON mt.id_truth_type = ts.id_truth_type 
           WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', 62, -37, 0.5)) = 1 '''

job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Additional asynchronous job query commands in case they're needed.

In [ ]:
# job.raise_if_error()
# job.abort()

Fetch the results into an astropy table called `results`.

In [ ]:
results = job.fetch_result().to_table()

Option to show results.

In [ ]:
# results

Print the number of SNe and the number of host galaxies in the area. Out of 1.7 million simulated objects, ~1000 are SNe.

In [ ]:
tx = np.where(results['truth_type'] == 3)[0]
print('The number of SNe in the area: ', len(tx))
del tx

tx = np.where(results['host_galaxy'] > 0)[0]
print('The number of SN host galaxies in the area: ', len(tx))
del tx

Store the list of host galaxy `id` in `host_ids`. It must be formatted as a string.

In [ ]:
tx = np.where(results['host_galaxy'] > 0)[0]
host_ids = np.asarray(results['host_galaxy'][tx], dtype='str')
del tx

How many of these host galaxies are matched to a simulated object in our test area (`nmatch`),
and were detected in a deepCoadd and are in the `Object` table (`ndetect`)?

While figuring this out, also create a list of the detected host galaxies' `id_truth_type` (temp1) and `match_objectId` (temp2).

In [ ]:
nmatch = 0
ndetect = 0

temp1 = []
temp2 = []

for i in range(len(host_ids)):
    tx = np.where(results['id'] == host_ids[i])[0]
    if len(tx) == 1:
        nmatch += 1
        if results['match_objectId'][tx] > 0:
            ndetect += 1
            temp1.append(results['id_truth_type'][tx[0]])
            temp2.append(results['match_objectId'][tx[0]])
    del tx

print('Number of host galaxies matched to the results table: ', nmatch)
print('Number of host galaxies that were detected and are in the Object table:', ndetect)
del nmatch, ndetect

host_id_truth_type = np.asarray(temp1, dtype='str')
host_match_objectId = np.asarray(temp2, dtype='int')
del temp1, temp2

There were 912 simulated galaxies in the test area that served as host galaxies AND were detected in a deepCoadd.

However, this does not mean that all 912 simulated SNe were detected and are in the `diaObject` table.
That is a question beyond the scope of this demo. See instead the "dia_match_truth.ipynb" notebook in the
"delegate-contributions-dp02/dia_exploration/" folder.

The end result is a list of the `id_truth_type` and `match_objectId` for galaxies in the test area which
were used as SN hosts in the DC2 simulation, and were also detected in a deepCoadd.

In [ ]:
print(host_id_truth_type[0])
print(host_match_objectId[0])
print(len(host_id_truth_type), len(host_match_objectId))